In [21]:
import pandas as pd
import scanpy as sc
import sys
import numpy as np
import scipy
import joblib

In [19]:
out_file_path = './'
data_dir = out_file_path + 'data/'
gene_signature_dir = out_file_path + 'gene_signatures/'

From DESeq2, look for enrichment of metabolism terms in DE genes

**read gene signatures**

In [22]:
mouse_filename = gene_signature_dir + 'metabolism_terms_mouse.pickle'
mouse_terms = joblib.load(mouse_filename)

In [23]:
for k,v in mouse_terms.items():
    print(f'{k}: {len(v)}')

KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS: 66
KEGG_MM_CITRATE_CYCLE: 33
KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY: 30
KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS: 34
KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM: 40
KEGG_MM_GALACTOSE_METABOLISM: 31
KEGG_MM_ASCORBATE_AND_ALDARATE_METABOLISM: 27
KEGG_MM_FATTY_ACID_BIOSYNTHESIS: 6
KEGG_MM_FATTY_ACID_METABOLISM: 48
KEGG_MM_SYNTHESIS_AND_DEGRADATION_OF_KETONE_BODIES: 11
KEGG_MM_STEROID_BIOSYNTHESIS: 17
KEGG_MM_PRIMARY_BILE_ACID_BIOSYNTHESIS: 16
KEGG_MM_STEROID_HORMONE_BIOSYNTHESIS: 58
KEGG_MM_OXIDATIVE_PHOSPHORYLATION: 142
KEGG_MM_PURINE_METABOLISM: 170
KEGG_MM_CAFFEINE_METABOLISM: 9
KEGG_MM_PYRIMIDINE_METABOLISM: 100
KEGG_MM_ALANINE_ASPARTATE_AND_GLUTAMATE_METABOLISM: 35
KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM: 39
KEGG_MM_CYSTEINE_AND_METHIONINE_METABOLISM: 39
KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_DEGRADATION: 53
KEGG_MM_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTHESIS: 2
KEGG_MM_LYSINE_BIOSYNTHESIS: 5
KEGG_MM_LYSINE_DEGRADATION: 48
KEGG_MM_ARGININE_AND

In [37]:
# read in DE genes
compass_data = pd.read_csv(data_dir + 'th_data/GSE162300_DFMO_RNA_est_counts.csv', sep = ',', index_col = 0) # this is library size normalized already
samples = list(set([i[:-9] for i in compass_data.columns]))

In [52]:
n_genes_total = len(compass_data.index)
n_genes_total

20817

In [38]:
samples

['Th17p_DFMO',
 'Th17n_Vehicle',
 'Th17n_DFMO',
 'iTreg_Vehicle',
 'Th17p_Vehicle',
 'iTreg_DFMO']

In [47]:
de_genes = {}
i =0 
for s1 in samples:
    for s2 in samples:
        if s1 != s2:
            de_genes[str(i)] = {'s1': s1, 's2': s2, 'res': pd.read_csv(data_dir + 'th_data/deseq_genes_%s_%s.csv' %(s1, s2), index_col = 0)}
            de_genes[str(i)]['res']['genes'] = de_genes[str(i)]['res'].index
            i+=1

In [49]:
from utils import hypergeometric_test, adjust_p_value_fdr

In [56]:
logfc_thresh = 1
p_thresh = 0.01

In [58]:
M = n_genes_total
hypergeom_res = {k: {} for k in de_genes.keys()}
for samp in de_genes.keys():
    df = de_genes[samp]['res']
    df = df[df['log2FoldChange'] > logfc_thresh]
    df = df[df['padj'] < p_thresh]
    df['genes'] = df['genes'].str.upper()
    df_genes = set(df['genes'])
    N = len(df) # length of DE genes
    for k,v in mouse_terms.items():
        n = len(v) # number of met genes
        x = len(df_genes.intersection(v))
        pct_overlap = x/(N+n)
        
        if pct_overlap > 0:
            p_val = hypergeometric_test(total_genes_expressed=M, n_genes_of_interest=n, 
                                        n_genes_picked=N, n_overlap=x)
            hypergeom_res[samp][k] =  p_val
        else:
            hypergeom_res[samp][k] = 0

In [73]:
hypergeom_df = pd.DataFrame.from_dict(hypergeom_res)
hypergeom_df['term'] = hypergeom_df.index
hypergeom_df = hypergeom_df.melt('term')
hypergeom_df['p'] = hypergeom_df['value']
hypergeom_df['padj'] = adjust_p_value_fdr(hypergeom_df['p'])
# add sample info
hypergeom_df['s1'] = None
hypergeom_df['s2'] = None
for k,v in de_genes.items():
    hypergeom_df.loc[hypergeom_df['variable'] == k, 's1'] = v['s1']
    hypergeom_df.loc[hypergeom_df['variable'] == k, 's2'] = v['s2']
hypergeom_df

,term,variable,value,p,padj,s1,s2
0,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
1,KEGG_MM_CITRATE_CYCLE,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
2,KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
3,KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS,0,0.695560,0.695560,1.000000,Th17p_DFMO,Th17n_Vehicle
4,KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM,0,0.753245,0.753245,1.000000,Th17p_DFMO,Th17n_Vehicle
...,...,...,...,...,...,...,...
2605,KEGG_MM_MTOR_SIGNALING_PATHWAY,29,0.102240,0.102240,0.737148,iTreg_DFMO,Th17p_Vehicle
2606,KEGG_MM_PI3K-AKT_SIGNALING_PATHWAY,29,0.079875,0.079875,0.148274,iTreg_DFMO,Th17p_Vehicle
2607,KEGG_MM_NOD-LIKE_RECEPTOR_SIGNALING_PATHWAY,29,0.422855,0.422855,0.508829,iTreg_DFMO,Th17p_Vehicle
2608,KEGG_MM_INSULIN_SIGNALING_PATHWAY,29,0.816993,0.816993,1.000000,iTreg_DFMO,Th17p_Vehicle


In [74]:
hypergeom_df[hypergeom_df['padj'] < 0.05]

,term,variable,value,p,padj,s1,s2
0,KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
1,KEGG_MM_CITRATE_CYCLE,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
2,KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
5,KEGG_MM_GALACTOSE_METABOLISM,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
6,KEGG_MM_ASCORBATE_AND_ALDARATE_METABOLISM,0,0.000000,0.000000,0.000000,Th17p_DFMO,Th17n_Vehicle
...,...,...,...,...,...,...,...
2593,KEGG_MM_FOLATE_BIOSYNTHESIS,29,0.000000,0.000000,0.000000,iTreg_DFMO,Th17p_Vehicle
2598,KEGG_MM_AMINOACYL-TRNA_BIOSYNTHESIS,29,0.000000,0.000000,0.000000,iTreg_DFMO,Th17p_Vehicle
2600,KEGG_MM_BIOSYNTHESIS_OF_UNSATURATED_FATTY_ACIDS,29,0.000000,0.000000,0.000000,iTreg_DFMO,Th17p_Vehicle
2601,KEGG_MM_METABOLIC_PATHWAYS,29,0.004408,0.004408,0.008356,iTreg_DFMO,Th17p_Vehicle


In [75]:
hypergeom_df.to_csv(out_file_path + 'compass_th_hypergeom_results.csv')

In [80]:
for k,v in de_genes.items():
    print(f"{k}: {v['s1']}, {v['s2']}")

0: Th17p_DFMO, Th17n_Vehicle
1: Th17p_DFMO, Th17n_DFMO
2: Th17p_DFMO, iTreg_Vehicle
3: Th17p_DFMO, Th17p_Vehicle
4: Th17p_DFMO, iTreg_DFMO
5: Th17n_Vehicle, Th17p_DFMO
6: Th17n_Vehicle, Th17n_DFMO
7: Th17n_Vehicle, iTreg_Vehicle
8: Th17n_Vehicle, Th17p_Vehicle
9: Th17n_Vehicle, iTreg_DFMO
10: Th17n_DFMO, Th17p_DFMO
11: Th17n_DFMO, Th17n_Vehicle
12: Th17n_DFMO, iTreg_Vehicle
13: Th17n_DFMO, Th17p_Vehicle
14: Th17n_DFMO, iTreg_DFMO
15: iTreg_Vehicle, Th17p_DFMO
16: iTreg_Vehicle, Th17n_Vehicle
17: iTreg_Vehicle, Th17n_DFMO
18: iTreg_Vehicle, Th17p_Vehicle
19: iTreg_Vehicle, iTreg_DFMO
20: Th17p_Vehicle, Th17p_DFMO
21: Th17p_Vehicle, Th17n_Vehicle
22: Th17p_Vehicle, Th17n_DFMO
23: Th17p_Vehicle, iTreg_Vehicle
24: Th17p_Vehicle, iTreg_DFMO
25: iTreg_DFMO, Th17p_DFMO
26: iTreg_DFMO, Th17n_Vehicle
27: iTreg_DFMO, Th17n_DFMO
28: iTreg_DFMO, iTreg_Vehicle
29: iTreg_DFMO, Th17p_Vehicle


In [64]:
hypergeom_res

{'0': {'KEGG_MM_GLYCOLYSIS_GLUCONEOGENESIS': 0,
  'KEGG_MM_CITRATE_CYCLE': 0,
  'KEGG_MM_PENTOSE_PHOSPHATE_PATHWAY': 0,
  'KEGG_MM_PENTOSE_AND_GLUCURONATE_INTERCONVERSIONS': 0.6955601115895058,
  'KEGG_MM_FRUCTOSE_AND_MANNOSE_METABOLISM': 0.7532452550186388,
  'KEGG_MM_GALACTOSE_METABOLISM': 0,
  'KEGG_MM_ASCORBATE_AND_ALDARATE_METABOLISM': 0,
  'KEGG_MM_FATTY_ACID_BIOSYNTHESIS': 0,
  'KEGG_MM_FATTY_ACID_METABOLISM': 0.8135442679435141,
  'KEGG_MM_SYNTHESIS_AND_DEGRADATION_OF_KETONE_BODIES': 0,
  'KEGG_MM_STEROID_BIOSYNTHESIS': 0,
  'KEGG_MM_PRIMARY_BILE_ACID_BIOSYNTHESIS': 0,
  'KEGG_MM_STEROID_HORMONE_BIOSYNTHESIS': 0.8686601784523652,
  'KEGG_MM_OXIDATIVE_PHOSPHORYLATION': 0.9581689594881504,
  'KEGG_MM_PURINE_METABOLISM': 0.23075233629023514,
  'KEGG_MM_CAFFEINE_METABOLISM': 0,
  'KEGG_MM_PYRIMIDINE_METABOLISM': 0.12983892860597496,
  'KEGG_MM_ALANINE_ASPARTATE_AND_GLUTAMATE_METABOLISM': 0.7060337929739788,
  'KEGG_MM_GLYCINE_SERINE_AND_THREONINE_METABOLISM': 0.744451473297975,
  '

# apply mann-whitney for DE analysis
> didn't find any genes

In [4]:
compass_data = pd.read_csv(data_dir + 'compass_expression.tsv', sep = '\t') # this is library size normalized already

In [5]:
adata = sc.read_csv(data_dir + 'compass_expression.tsv', delimiter = '\t')
adata = adata.transpose()
adata.obs['sample'] = adata.obs.index
adata

AnnData object with n_obs × n_vars = 40 × 40152
    obs: 'sample'

In [6]:
adata

AnnData object with n_obs × n_vars = 40 × 40152
    obs: 'sample'

In [8]:
adata.obs

,sample
WT-d_S141_L007_R1_001,WT-d_S141_L007_R1_001
Ob-DHA-e_S154_L007_R1_001,Ob-DHA-e_S154_L007_R1_001
WT-1012-c_S135_L007_R1_001,WT-1012-c_S135_L007_R1_001
Ob-DHA-b_S132_L007_R1_001,Ob-DHA-b_S132_L007_R1_001
Ob-EPA-a_S122_L007_R1_001,Ob-EPA-a_S122_L007_R1_001
Ob-c_S136_L007_R1_001,Ob-c_S136_L007_R1_001
Ob-1012-c_S140_L007_R1_001,Ob-1012-c_S140_L007_R1_001
WT-EPA-e_S151_L007_R1_001,WT-EPA-e_S151_L007_R1_001
WT-DHA-d_S143_L007_R1_001,WT-DHA-d_S143_L007_R1_001
WT-DHA-b_S128_L007_R1_001,WT-DHA-b_S128_L007_R1_001


**differential gene expression**

In [45]:
# import random
# adata.obs['louvain'] = ['clust_' + str(random.sample([1,2,3], 1)[0]) for i in range(len(adata.obs))]
# adata.obs['louvain'] = adata.obs['louvain'].astype('category')
# adata.layers['raw'] = adata.X
# adata.layers['scT'] = adata.X

In [106]:
# write simple DE test based off scRNA-methods
# base this off of previous differential expression script
# can't use DeSEq2 because input requires raw counts
def get_count_data(adata, clustering_name = 'louvain', cluster1 = '0', layer = 'lib_norm'):
    cluster1 = [cluster1]
    cluster2 = [str(c) for c in adata.obs[clustering_name].unique()]
    x = adata.layers[layer][adata.obs[clustering_name].isin(cluster1)]
    y = adata.layers[layer][adata.obs[clustering_name].isin(cluster2)]
    return x,y

def get_de_genes(adata, clustering_name = 'sample', cluster = '0', log2FC_threshold = 0.5, padj_threshold = 0.05, layer = 'lib_norm'):
    x,y = get_count_data(adata, clustering_name = clustering_name, cluster1 = cluster, layer = layer)
    corrector_value = np.median(np.squeeze(adata.layers[layer].mean(axis = 0)))
    
    x_mean = np.squeeze(np.asarray((np.sum(x, 0) / x.shape[0])))
    y_mean = np.squeeze(np.asarray((np.sum(y, 0) / y.shape[0])))
    
    log2FC = np.log2((x_mean + corrector_value) / (y_mean + corrector_value))
    log2FC_high = np.where(abs(log2FC) > log2FC_threshold)[0]
    
    pvals = [scipy.stats.mannwhitneyu(x[:, i], y[:, i])[1] for i in log2FC_high]
    pvals_corrected = [p * len(log2FC_high) for p in pvals]
    
    log2FC_data = {'log2FC': log2FC[log2FC_high].tolist(),
                   'x_mean': x_mean[log2FC_high], 'y_mean': y_mean[log2FC_high],
                   'pval': pvals, 'padj': pvals_corrected}
    
    log2FC_data = pd.DataFrame(log2FC_data, index=adata.var_names[log2FC_high])
    log2FC_data = log2FC_data[log2FC_data['padj'] < padj_threshold].sort_values(by='log2FC', ascending=False)
    
    return(log2FC_data)

def get_de_genes_each_vs_all(adata, clustering_name = 'sample', log2FC_threshold = 0.5, padj_threshold = 0.05, layer = 'lib_norm'):
    all_logFC_data = {}
    for cluster in adata.obs[clustering_name].unique():
        print(cluster)
        all_logFC_data[cluster] = get_de_genes(adata, clustering_name = clustering_name, cluster = cluster, log2FC_threshold = log2FC_threshold, 
                                             padj_threshold = padj_threshold, layer = layer)
        adata.uns['%s_logFC' % (clustering_name)] = all_logFC_data
    return all_logFC_data

In [75]:
de_logFC_thresh = 0.5
de_p_thresh = 0.01

In [84]:
adata.layers['lib_norm'] = adata.X

In [107]:
de_genes = get_de_genes_each_vs_all(adata, 'sample', log2FC_threshold=de_logFC_thresh, padj_threshold=de_p_thresh)

WT-d_S141_L007_R1_001
Ob-DHA-e_S154_L007_R1_001
WT-1012-c_S135_L007_R1_001
Ob-DHA-b_S132_L007_R1_001
Ob-EPA-a_S122_L007_R1_001
Ob-c_S136_L007_R1_001
Ob-1012-c_S140_L007_R1_001
WT-EPA-e_S151_L007_R1_001
WT-DHA-d_S143_L007_R1_001
WT-DHA-b_S128_L007_R1_001
WT-EPA-a_S117_L007_R1_001
WT-1012-d_S145_L007_R1_001
Ob-1012-d_S149_L007_R1_001
WT-DHA-a_S118_L007_R1_001
Ob-911-c_S139_L007_R1_001
Ob-d_S146_L007_R1_001
WT-EPA-b_S127_L007_R1_001
Ob-911-a_S124_L007_R1_001
Ob-b_S131_L007_R1_001
WT-1012-b_S130_L007_R1_001
Ob-EPA-c_S137_L007_R1_001
Ob-EPA-e_S153_L007_R1_001
Ob-1012-b_S133_L007_R1_001
WT-911-b_S129_L007_R1_001
WT-911-a_S119_L007_R1_001
WT-DHA-c_S134_L007_R1_001
WT-EPA-d_S142_L007_R1_001
Ob-a_S121_L007_R1_001
Ob-DHA-c_S138_L007_R1_001
WT-1012-a_S120_L007_R1_001
Ob-1012-a_S125_L007_R1_001
Ob-DHA-a_S123_L007_R1_001
WT-e_S150_L007_R1_001
Ob-911-e_S155_L007_R1_001
WT-a_S116_L007_R1_001
Ob-911-d_S148_L007_R1_001
WT-1012-e_S152_L007_R1_001
WT-911-d_S144_L007_R1_001
Ob-DHA-d_S147_L007_R1_001
WT-b_

In [109]:
de_genes

{'WT-d_S141_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-DHA-e_S154_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-1012-c_S135_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-DHA-b_S132_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-EPA-a_S122_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-c_S136_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'Ob-1012-c_S140_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-EPA-e_S151_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-DHA-d_S143_L007_R1_001': Empty DataFrame
 Columns: [log2FC, x_mean, y_mean, pval, padj]
 Index: [],
 'WT-DHA-b_S128_L007_R1_001': Empty DataFrame
 Columns: [log2

In [10]:
t = pd.read_csv('/Genomics/pritykinlab/amanda/other/metabolic_analysis/metabolic_analysis/data/th_data/GSE162300_DFMO_RNA_est_counts.csv')

In [11]:
t

,symbol,Th17p_Vehicle_WT1_run1,Th17p_Vehicle_WT1_run2,Th17n_Vehicle_WT1_run1,Th17n_Vehicle_WT1_run2,iTreg_Vehicle_WT1_run1,iTreg_Vehicle_WT1_run2,Th17p_DFMO_WT1_run1,Th17p_DFMO_WT1_run2,Th17n_DFMO_WT1_run1,...,Th17n_Vehicle_WT3_run1,Th17n_Vehicle_WT3_run2,iTreg_Vehicle_WT3_run1,iTreg_Vehicle_WT3_run2,Th17p_DFMO_WT3_run1,Th17p_DFMO_WT3_run2,Th17n_DFMO_WT3_run1,Th17n_DFMO_WT3_run2,iTreg_DFMO_WT3_run1,iTreg_DFMO_WT3_run2
0,0610007P14RIK,787.00,870.00,708.00,845.00,570.00,624.00,786.0,846.00,864.00,...,1183.00,1332.00,794.00,853.00,928.00,1045.00,1120.00,1163.00,710.00,723.00
1,0610009B22RIK,93.00,98.00,66.00,70.00,87.90,100.00,123.0,128.00,106.00,...,94.00,76.00,109.00,145.00,121.00,131.00,131.00,137.78,137.00,148.00
2,0610009L18RIK,2.00,2.00,2.00,1.00,3.00,4.00,4.0,5.00,3.00,...,6.00,3.00,0.00,0.00,5.00,11.00,3.00,4.00,4.00,5.00
3,0610009O20RIK,177.00,206.00,148.00,170.00,194.00,159.00,168.0,172.00,161.00,...,249.00,284.00,202.00,254.98,216.00,228.98,212.00,226.00,258.97,246.00
4,0610010F05RIK,24.00,23.00,23.00,25.00,41.00,43.00,32.0,22.00,32.00,...,46.00,39.00,30.00,40.00,44.00,55.00,24.00,31.00,20.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20812,ZYG11A,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
20813,ZYG11B,302.34,291.59,215.25,245.82,229.23,251.62,169.6,210.45,180.24,...,359.07,375.34,386.87,405.60,242.78,271.74,276.50,265.54,271.90,273.43
20814,ZYX,250.00,320.00,328.00,422.00,177.00,226.00,288.0,331.00,378.00,...,586.00,627.00,279.00,290.00,420.00,484.00,533.00,629.00,200.00,232.00
20815,ZZEF1,220.00,230.00,145.00,177.00,235.00,238.00,154.0,199.00,175.00,...,203.00,261.00,258.00,283.00,243.00,262.00,275.00,274.00,325.00,316.00


In [14]:
t = pd.read_csv('/Genomics/pritykinlab/amanda/other/metabolic_analysis/metabolic_analysis/data/th_data/74833_samps/GSM1943520_single-cells_Tgfb1_IL6-IL17A_POS_48hr_140519_T16_IL17A_48h_SC90_408.genes.fpkm_tracking', sep = '\t')

In [17]:
t['FPKM'].unique()

array([   0.    ,   25.5533,  223.804 , ...,   84.2759,   24.3575,
       4556.79  ])